In [1]:
 # De ley
import pandas as pd 
import numpy as np 
from datetime import date

#Normalizacion de palabras 
from thefuzz import fuzz, process

# Graficos 
import seaborn as sns
import matplotlib.pyplot as plt 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

#prepocesamiento de datos
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.decomposition import PCA
import category_encoders as ce


# division entranamiento-validacion
from sklearn.model_selection import train_test_split 
#clustering 
from sklearn.cluster import KMeans

C:\Users\XMX5634\anaconda3\lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
C:\Users\XMX5634\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Funciones

In [2]:
def tabla_frecuencias(df,col):
    #construir una tabla de frecuencias. 
    
    rating_counts = (df.groupby(col).agg(frec_absoluta=(col, "count"))).sort_values(['frec_absoluta'], ascending = False)
    
    rating_counts["frec_absoluta_acumulada"] = rating_counts["frec_absoluta"].cumsum()

    
    rating_counts['frec_relativa'] = rating_counts['frec_absoluta'].apply(lambda x: x/rating_counts['frec_absoluta'].sum())
    rating_counts['frec_relativa(%)'] = rating_counts['frec_relativa'].apply(lambda x: x*100 )

    
    
    rating_counts['frec_relativa_acumulada'] = rating_counts['frec_absoluta_acumulada'].apply(lambda x: x/rating_counts['frec_absoluta'].sum())  
    rating_counts['frec_relativa_acumulada(%)'] = rating_counts['frec_relativa_acumulada'].apply(lambda x: x*100)
    
    return rating_counts

In [3]:
def tabla_intervalo_frecuencias(df,col,long_intervalos):
    #construir una tabla de frecuencias. 
    aux = df
    bins = list(range(int(min(df[col])),int(max(df[col]))+1,long_intervalos))

    aux["intervalos"] = pd.cut(df[col], bins=bins)


    rating_counts = (df.groupby("intervalos").agg(frec_absoluta=(col, "count")))
    
    rating_counts["frec_absoluta_acumulada"] = rating_counts["frec_absoluta"].cumsum()

    
    rating_counts['frec_relativa'] = rating_counts['frec_absoluta'].apply(lambda x: x/rating_counts['frec_absoluta'].sum())
    rating_counts['frec_relativa(%)'] = rating_counts['frec_relativa'].apply(lambda x: x*100 )

    
    
    rating_counts['frec_relativa_acumulada'] = rating_counts['frec_absoluta_acumulada'].apply(lambda x: x/rating_counts['frec_absoluta'].sum())  
    rating_counts['frec_relativa_acumulada(%)'] = rating_counts['frec_relativa_acumulada'].apply(lambda x: x*100)

    
    return rating_counts

In [4]:
def conv(x):
    meses = {'enero':'01','febrero':'02','marzo':'03','abril':'04','mayo':'05','junio':'06','julio':'07','agosto':'08',
            'septiembre':'09','octubre':'10','noviebre':'11','diciembre':'12'}
    if x == None:
        return 0
    for mes in meses:
        if mes == x:
            return meses[x]

In [5]:
def con_categ_to_columns(df,col,col_fecha):
    data = pd.DataFrame({})
    uniques = df[col].unique()
    df = df[[col_fecha,col]]
    for unique in uniques:
        aux = df[df[col] == unique]
        aux2 = (aux.groupby(col_fecha).agg(count=(col, "count"))).sort_values([col_fecha], ascending = True)
        data[unique] = aux2['count'] 
    data['fecha'] = aux2.index
    return data

In [6]:
def hora2(x):
    try:
        x = str(x)
        x1 = x[:x.index(':')+1].replace(':','.')
        x2 = x[x.index(':')+1:].replace(':','')
        x = x1 + x2
        x = float(x)
        return round(x)
        
    except:
        return '0'

In [7]:
def hora(x):
    normalizar = {12:12,1:13,2:14,3:15,4:16,5:17,6:18,7:19,8:20,9:21,10:22,11:23}
    if x == None:
        return 'DESC'
    
    else:
        x = str(x)
        try:
            x = str(x)
            x1 = x[:x.index(':')+1].replace(':','.')
            x2 = x[x.index(':')+1:].replace(':','')
            x = x1 + x2
            if 'p' in x:
                x = x.split('p')[0]
                x = float(x)
                ent = int(x)
                dec = x - ent
                x = normalizar[ent] + dec
                if 12.00 <= x and x < 19.00:
                    return 'TARDE'
                elif 19.00 <= x and x <= 23.59:
                    return 'NOCHE'
            else:
                x = x.split('a')[0]
                x = float(x)
                ent = int(x)
                dec = x - ent
                x = normalizar[ent] + dec
                if 12 <= x and x < 18.00:
                    return 'MADRUGADA'
                elif  18 <= x and x <= 23.59:
                    return 'MAÑANA'
        except:
            return '0'

In [8]:
def num_indentidades(df):
    dicc1 = {}
    dicc2 = {}
    for id_ in df.fillna({'identidad':'desconocido'}).identidad.unique():
        dicc1[id_] = len(str(id_).split(' '))
        dicc2[id_] = str(id_).split(' ')

    print('la longitud maxima es de ' +str(len(dicc2[max(dicc1, key=dicc1.get)]))+': '+ max(dicc1, key=dicc1.get))

In [9]:
def f_columns(df):
    dicc1 = {}
    df = df.fillna({'identidad':'DESCONOCIDO'})
    aux = [str(x).split(' ') for x in df['identidad'].unique()]
    max_ = max(len(x) for x in aux)

    i=0
    for lista in aux:
        while len(lista) < max_:
            lista.append(None)
        dicc1[df['identidad'].unique()[i]] = lista
        i+=1

    df['aux'] = df['identidad'].map(dicc1)
    df['len_aux'] = df['aux'].apply(lambda x: len(x))
    df['len_identidad'] = df['identidad'].apply(lambda x: len(str(x).split(' ')))

    for i in range(max_):
        df['identidad'+str(i+1)] = df['aux'].apply(lambda x: x[i])
        
    return df

In [10]:
def algoritm(row):
    string = row['identidad_split']
    prueba = row['concatenacion']
    d=0
    words = []
    for idx in range(len(prueba)):
        u1 = 'DESCONOCIDO'
        value = int(prueba[idx])    
        if idx == 0:
            if  value != 0:
                try:
                    u1 = '_'.join(string[0:value])
                    d = len(string[0:value])
                except:
                    pass

        else:
            if value != 0:
                try:
                    u1 = '_'.join(string[d:d+value])
                    d = d + len(string[d:d+value])
                except:
                    pass
        words.append(u1)
    return words

In [11]:
def algoritm(row):
    string = row['identidad_split']
    prueba = row['concatenacion']
    d=0
    words = []
    for idx in range(len(prueba)):
        u1 = 'DESCONOCIDO'
        value = int(prueba[idx])    
        if idx == 0:
            if  value != 0:
                try:
                    u1 = '_'.join(string[0:value])
                    d = len(string[0:value])
                except:
                    pass

        else:
            if value != 0:
                try:
                    u1 = '_'.join(string[d:d+value])
                    d = d + len(string[d:d+value])
                except:
                    pass
        words.append(u1)
    return words

In [12]:
def algoritm_edad_lesionados(row):
    string = row['edad_lesionado_split']
    prueba = row['concatenacion_edad_lesionados']
    d=0
    words = []
    for idx in range(len(prueba)):
        u1 = '0'
        value = int(prueba[idx])    
        if idx == 0:
            if  value != 0:
                try:
                    u1 = '_'.join(string[0:value])
                    d = len(string[0:value])
                except:
                    pass

        else:
            if value != 0:
                try:
                    u1 = '_'.join(string[d:d+value])
                    d = d + len(string[d:d+value])
                except:
                    pass
        words.append(u1)
    return words

In [13]:
def algoritm_edad_occisos(row):
    string = row['edad_occiso_split']
    prueba = row['concatenacion_edad_occisos']
    d=0
    words = []
    for idx in range(len(prueba)):
        u1 = '0'
        value = int(prueba[idx])    
        if idx == 0:
            if  value != 0:
                try:
                    u1 = '_'.join(string[0:value])
                    d = len(string[0:value])
                except:
                    pass

        else:
            if value != 0:
                try:
                    u1 = '_'.join(string[d:d+value])
                    d = d + len(string[d:d+value])
                except:
                    pass
        words.append(u1)
    return words

In [14]:
def levenshteinDistance(A, B):
    N, M = len(A), len(B)
    # Create an array of size NxM
    dp = [[0 for i in range(M + 1)] for j in range(N + 1)]

    # Base Case: When N = 0
    for j in range(M + 1):
        dp[0][j] = j
    # Base Case: When M = 0
    for i in range(N + 1):
        dp[i][0] = i
    # Transitions
    for i in range(1, N + 1):
        for j in range(1, M + 1):
            if A[i - 1] == B[j - 1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = 1 + min(
                    dp[i-1][j], # Insertion
                    dp[i][j-1], # Deletion
                    dp[i-1][j-1] # Replacement
                )

    return dp[N][M]

def comparacion(A, target):
    targets = [target.upper() for i in range(len(A))]
    dicc = {}
    for i in range(len(A)):
        u = levenshteinDistance(A[i].upper(), targets[i])
        dicc[A[i]] = u
        
    return min(dicc, key=dicc.get)
        

# TRATAMIENTO

## Limpieza General

In [15]:
ruta_git = 'https://raw.githubusercontent.com/RiemanNClav/Hechos-de-Transito-registrados-por-la-SSC/master/hechos_transito.csv'
ruta_local = 'C:/Users/XMX5634/Documents/Hechos-de-Transito-registrados-por-la-SSC/hechos_transito.csv'
df = pd.read_csv(ruta_local)

C:\Users\XMX5634\AppData\Local\Temp/ipykernel_1956/1823139733.py:3: DtypeWarning: Columns (4,17,18,19,20,21,22,24,25,26,49,53,58,59,60,61,62,63,64,66,67,68,69,70,71,74,75,76,77,78,79,80,83,84,85,87) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta_local)


In [16]:
id_ = ['no_folio']
periodos = ['fecha_evento', 'mes', 'hora', 'dia']
caracteristicas_evento = ['condicion', 'tipo_de_evento', 'prioridad', 'diagnostico_lesionado', 'diagnostico_occiso', 'origen']
caracteristicas_terrestres = ['punto_1', 'punto_2', 'colonia', 'alcaldia', 'tipo_de_interseccion',
                              'interseccion_semaforizada', 'clasificacion_de_la_vialidad', 'sentido_de_circulacion', 'sector',
                              'coordenada_x', 'coordenada_y']
caracteristicas_objetos = ['tipo_de_vehiculo_1', 'marca_de_vehiculo_1','color_vehiculo_1',
                           'tipo_de_vehiculo_2', 'marca_de_vehiculo_2','color_vehiculo_2',
                                'tipo_de_vehiculo_3', 'marca_de_vehiculo_3','color_vehiculo_3',
                           'tipo_de_vehiculo_4', 'marca_de_vehiculo_4',
                           'unidad_medica_de_apoyo']

caracteristicas_involucrados = ['edad_occiso','se_desconoce_occiso', 'total_occisos', 'occisos_masculinos', 'occisos_femeninos','occiso_se_desconoce', 'peaton_occiso', 'ciclista_occiso', 'pasajero_occiso', 'motociclista_occiso', 'conductor_occiso',
                                'lesiones', 'edad_lesionado', 'se_desconoce_lesionado', 'total_lesionados', 'lesionados_masculinos', 'lesionados_femeninos', 'lesionado_se_desconoce', 'peaton_lesionado', 'ciclista_lesionado', 'pasajero_lesionado', 'motociclista_lesionado', 'conductor_lesionado']

variadas = ['identidad', 'hospital']

df = df[id_ + periodos + caracteristicas_evento + caracteristicas_terrestres + caracteristicas_objetos + caracteristicas_involucrados + variadas]

In [17]:
df['hora'] = df['hora'].apply(lambda x: hora2(x)).astype(int)
df = df[df['hora'].isin([i for i in range(1,24)])]
df['year'] = pd.to_datetime(df['fecha_evento']).dt.to_period('Y')
df = df.fillna({'diagnostico_occiso': 'DESCONOCIDO', 'diagnostico_lesionado': 'DESCONOCIDO', 'origen': 'DESCONOCIDO'})

In [18]:
dias = ['lunes', 'martes', 'miercoles', 'jueves', 'viernes', 'sabado', 'domingo']
dicc={}
for dia in df.dia.unique():
    dicc[dia] = comparacion(dias, dia)
    df.loc[df['dia'] == dia, 'dia'] = dicc[dia]

In [19]:
meses = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']
dicc={}
for mes in df.mes.unique():
    dicc[mes] = comparacion(meses, mes)
    df.loc[df['mes'] == mes, 'mes'] = dicc[mes]

In [20]:
colores = ["rojo", "naranja", "amarillo", "verde", "azul", "morado", "rosa", "negro", "blanco", "gris", "marrón"]
dicc={}
for vehiculo in [str(x) for x in df.color_vehiculo_1.unique()]:
    dicc[vehiculo] = comparacion(colores, vehiculo)
    df.loc[df['color_vehiculo_1'] == vehiculo, 'color_vehiculo_1'] = dicc[vehiculo]

In [21]:
color = df.color_vehiculo_1.unique()
dicc=  {}
for value in df.color_vehiculo_2.unique():
    new_color = process.extract(str(value), color, limit=1)[0][0]
    dicc[value] = new_color
    
df['aux'] = df['color_vehiculo_2'].map(dicc)
df = df.drop(['color_vehiculo_2'],axis=1)
df.rename(columns = {'aux':'color_vehiculo_2'}, inplace=True)

In [22]:
df.loc[df['identidad'] == 'PASAJERA DEL TAXI', 'identidad'] = 'PASAJERA_DEL_TAXI'
df.loc[df['edad_occiso'] == 'SD', 'edad_occiso'] = '0'
df.loc[df['edad_lesionado'] == 'SD', 'edad_lesionado'] = '0'

In [23]:
df = df.fillna({'edad_occiso':'0', 'edad_lesionado':'0'})

In [24]:
(df.isnull().sum()/len(df)) * 100

no_folio                         0.000938
fecha_evento                     0.000000
mes                              0.000000
hora                             0.000000
dia                              0.000000
condicion                       65.986790
tipo_de_evento                   0.000000
prioridad                        0.000000
diagnostico_lesionado            0.000000
diagnostico_occiso               0.000000
origen                           0.000000
punto_1                          0.000000
punto_2                          0.000000
colonia                          0.000000
alcaldia                         0.000000
tipo_de_interseccion             0.000000
interseccion_semaforizada       31.957631
clasificacion_de_la_vialidad    31.957631
sentido_de_circulacion          31.957631
sector                           0.000000
coordenada_x                     0.005629
coordenada_y                     0.003753
tipo_de_vehiculo_1               0.000000
marca_de_vehiculo_1             30

In [25]:
df['concatenacion'] = df['occisos_femeninos'].astype(str) + df['occisos_masculinos'].astype(str) + df['occiso_se_desconoce'].astype(str) + df['lesionados_femeninos'].astype(str) + df['lesionados_masculinos'].astype(str) + df['lesionado_se_desconoce'].astype(str) 

In [26]:
df['concatenacion_edad_occisos'] = df['occisos_femeninos'].astype(str) + df['occisos_masculinos'].astype(str) + df['occiso_se_desconoce'].astype(str) 

In [27]:
df['concatenacion_edad_lesionados'] = df['lesionados_femeninos'].astype(str) + df['lesionados_masculinos'].astype(str) + df['lesionado_se_desconoce'].astype(str) 

In [28]:
u = df[df['total_lesionados'] > 4]
u[['lesionados_femeninos', 'lesionados_masculinos', 'lesionado_se_desconoce', 'concatenacion_edad_lesionados', 'edad_lesionado']]

,lesionados_femeninos,lesionados_masculinos,lesionado_se_desconoce,concatenacion_edad_lesionados,edad_lesionado
328,5,0,0,500,38 45 23 34 55
552,2,4,0,240,26 29 29 24 23 32
685,2,3,0,230,18 26 20 SD SD
824,15,0,0,1500,SD SD SD SD SD SD SD SD SD SD SD SD SD SD SD
989,1,4,0,140,28 30 2 9 8
...,...,...,...,...,...
106986,4,4,0,440,15 21 33 38 10 13 22 24
107013,0,6,0,060,32 52 27 50 34 28
107022,3,2,0,320,34 31 31 35 56
107080,3,3,0,330,35 46 49 14 36 53


In [29]:
u = df[df['total_occisos'] > 1]
u[['occisos_femeninos', 'occisos_masculinos', 'occiso_se_desconoce', 'concatenacion_edad_occisos', 'edad_occiso']]

,occisos_femeninos,occisos_masculinos,occiso_se_desconoce,concatenacion_edad_occisos,edad_occiso
1616,1,1,0,110,0
2198,0,2,0,020,0
2614,1,1,0,110,0
3212,0,2,0,020,0
4163,1,1,0,110,0
...,...,...,...,...,...
109013,0,0,0,000,0
109058,0,0,0,000,ITALIKA
109193,0,0,0,000,0
109410,0,1,0,010,URVAN


# ----------------------------------------

## Limpieza de edades

### Lesionados

In [30]:
values_lesionados_letters = []
values_lesionados_comp = []
new_values_lesionados_letters = []
for col in df.edad_lesionado.unique(): 
    if len(col.strip().replace(',', ' ').split(' ')) == 1:
        try:
            col = int(col)
        except:
            #valores con solo una posición 
            values_lesionados_letters.append(col)
            try:
                value = re.search(r'[1-9][A-Z]$', col)
                period = value[0][-1].upper()
                if period == 'M' or period == 'Q':
                    col_ = int(value[0][0:-1]) / 10
                    new_values_lesionados_letters.append(str(col_))
            except:
                new_values_lesionados_letters.append(col.replace(''.join(col.split(' ')), '0'))
                    
    else:
        try:
            u = [int(x) for x in col.split(' ')]
        except:
            values_lesionados_comp.append(col)

In [31]:
dicc1 = dict(zip(values_lesionados_letters,new_values_lesionados_letters))

In [32]:
new_values_lesionados_comp = []
for value in values_lesionados_comp:
    value = value.replace('SD','0')
    comparar = value.split(' ')
    for i in range(len(comparar)):
        try:
            value_ = re.search(r'[1-9][A-Z]$', comparar[i])[0]
            period = value_[-1].upper()
            if period in ['M', 'Q']:
                comparar[i] = str(int(value_[0:-1]) / 10)
        except:
            try:
                ent = int(comparar[i])
            except:
                comparar[i] = '0'
        value = ' '.join(comparar)
    new_values_lesionados_comp.append(value)
    

In [33]:
dicc2 = dict(zip(values_lesionados_comp,new_values_lesionados_comp))

In [34]:
edades = [str(i) for i in range(0,111)]
dicc3 = dict(zip(edades,edades))

In [35]:
dicc1.update(dicc2)
dicc1.update(dicc3)

In [36]:
df['edad_lesionado_aux'] = df['edad_lesionado'].map(dicc1)

In [37]:
no_nulos = df[~(df['edad_lesionado_aux'].isnull())]
no_nulos = no_nulos.drop(['edad_lesionado'],axis=1)
no_nulos.rename(columns = {'edad_lesionado_aux':'edad_lesionado'}, inplace=True)

In [38]:
nulos = df[df['edad_lesionado_aux'].isnull()]
nulos = nulos.drop(['edad_lesionado_aux'], axis=1)

In [39]:
df = pd.concat([nulos,no_nulos], axis = 0, ignore_index = True)

### Occisos

In [40]:
values_occisos_letters = []
values_occisos_comp = []
new_values_occisos_comp = []
new_values_occisos_letters = []

for col in df.edad_occiso.unique(): 
    if len(str(col).split(' ')) == 1:
        values_occisos_letters.append(col)
        try:
            col_ = str(int(float(col)))
            new_values_occisos_letters.append(str(col_))
        except:
            #valores con solo una posición 
            col_ = '0'
            new_values_occisos_letters.append(col_)
    else:
        try:
            u = [int(x) for x in col.split(' ')]
        except:
            values_occisos_comp.append(col)
            col = col.replace('SD','0')
            new_values_occisos_comp.append('0')

In [41]:
dict1 = dict(zip(values_occisos_letters,new_values_occisos_letters))
dict2 = dict(zip(values_occisos_comp,new_values_occisos_comp))
dict1.update(dict2)

In [42]:
df['edad_occiso_aux'] = df['edad_occiso'].map(dict1)
no_nulos = df[~(df['edad_occiso_aux'].isnull())]
no_nulos = no_nulos.drop(['edad_occiso'],axis=1)
no_nulos.rename(columns = {'edad_occiso_aux':'edad_occiso'}, inplace=True)

In [43]:
nulos = df[df['edad_occiso_aux'].isnull()]
nulos = nulos.drop(['edad_occiso_aux'], axis=1)

In [44]:
len(no_nulos) + len(nulos) == len(df)

True

In [45]:
df = pd.concat([nulos, no_nulos], axis = 0, ignore_index = True)

In [46]:
df = f_columns(df)

In [47]:
df['identidad_split'] = df['identidad'].apply(lambda x: str(x).split(' '))

In [48]:
num_indentidades(df)

la longitud maxima es de 4: CONDUCTOR PASAJERO PASAJERO PASAJERO


In [49]:
df_1 = df[df['len_identidad'] == df['total_lesionados'] + df['total_occisos']]
proporcion = (len(df_1) / len(df)) * 100
print(proporcion)
print(len(df_1))

85.62596164671446
91267


In [50]:
df_2 = df[~(df['len_identidad'] == df['total_lesionados'] + df['total_occisos'])]
proporcion = (len(df_2) / len(df)) * 100
print(proporcion)
print(len(df_2))

14.374038353285549
15321


# ---------------------------------------------

## Limpieza de DataFrames
### df_1

In [51]:
df_1['edad_occiso'] = df_1['edad_occiso'].astype(str)
df_1['edad_lesionado'] = df_1['edad_lesionado'].astype(str)

C:\Users\XMX5634\AppData\Local\Temp/ipykernel_1956/252749915.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['edad_occiso'] = df_1['edad_occiso'].astype(str)
C:\Users\XMX5634\AppData\Local\Temp/ipykernel_1956/252749915.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['edad_lesionado'] = df_1['edad_lesionado'].astype(str)


In [52]:
for edad in df_1.edad_lesionado.unique():
        if len(edad.split(' ')) == 1:
            try:
                edad_aux = float(edad)
            except:
                edad_aux = float(edad[0])/100
                
            df_1.loc[df_1['edad_lesionado'] == edad, 'edad_lesionado'] = str(edad_aux)
                

In [53]:
df_1['edad_lesionado_split'] = df_1['edad_lesionado'].apply(lambda x: x.split(' '))
df_1['edad_occiso_split'] = df_1['edad_occiso'].astype(str).apply(lambda x: x.split(' '))

C:\Users\XMX5634\AppData\Local\Temp/ipykernel_1956/4027744053.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['edad_lesionado_split'] = df_1['edad_lesionado'].apply(lambda x: x.split(' '))
C:\Users\XMX5634\AppData\Local\Temp/ipykernel_1956/4027744053.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['edad_occiso_split'] = df_1['edad_occiso'].astype(str).apply(lambda x: x.split(' '))


In [54]:
df_1["algoritm"] = df_1.apply(algoritm, axis=1)

C:\Users\XMX5634\AppData\Local\Temp/ipykernel_1956/4197099403.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1["algoritm"] = df_1.apply(algoritm, axis=1)


In [55]:
df_1["algoritm_edad_occisos"] = df_1.apply(algoritm_edad_occisos, axis=1)

C:\Users\XMX5634\AppData\Local\Temp/ipykernel_1956/1773795021.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1["algoritm_edad_occisos"] = df_1.apply(algoritm_edad_occisos, axis=1)


In [56]:
cols = ['edad_occiso_fem','edad_occiso_masc','edad_occiso_desc']
for i in range(len(cols)):
    df_1[cols[i]] = df_1['algoritm_edad_occisos'].apply(lambda x: x[i])

C:\Users\XMX5634\AppData\Local\Temp/ipykernel_1956/3546912541.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1[cols[i]] = df_1['algoritm_edad_occisos'].apply(lambda x: x[i])
C:\Users\XMX5634\AppData\Local\Temp/ipykernel_1956/3546912541.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1[cols[i]] = df_1['algoritm_edad_occisos'].apply(lambda x: x[i])
C:\Users\XMX5634\AppData\Local\Temp/ipykernel_1956/3546912541.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [57]:
df_1["algoritm_edad_lesionados"] = df_1.apply(algoritm_edad_lesionados, axis=1)

C:\Users\XMX5634\AppData\Local\Temp/ipykernel_1956/1020833671.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1["algoritm_edad_lesionados"] = df_1.apply(algoritm_edad_lesionados, axis=1)


In [58]:
cols = ['edad_lesionados_fem','edad_lesionados_masc','edad_lesionados_desc']
for i in range(len(cols)):
    df_1[cols[i]] = df_1['algoritm_edad_lesionados'].apply(lambda x: x[i])

C:\Users\XMX5634\AppData\Local\Temp/ipykernel_1956/3622368339.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1[cols[i]] = df_1['algoritm_edad_lesionados'].apply(lambda x: x[i])
C:\Users\XMX5634\AppData\Local\Temp/ipykernel_1956/3622368339.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1[cols[i]] = df_1['algoritm_edad_lesionados'].apply(lambda x: x[i])
C:\Users\XMX5634\AppData\Local\Temp/ipykernel_1956/3622368339.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [59]:
cols = ['occisos_fem','occisos_masc','occisos_desc','lesionados_fem','lesionados_masc','lesionados_desc']
for i in range(len(cols)):
    df_1[cols[i]] = df_1['algoritm'].apply(lambda x: x[i])

C:\Users\XMX5634\AppData\Local\Temp/ipykernel_1956/2570950552.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1[cols[i]] = df_1['algoritm'].apply(lambda x: x[i])
C:\Users\XMX5634\AppData\Local\Temp/ipykernel_1956/2570950552.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1[cols[i]] = df_1['algoritm'].apply(lambda x: x[i])
C:\Users\XMX5634\AppData\Local\Temp/ipykernel_1956/2570950552.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

In [60]:
dicc = {}
cols_edades = ['edad_lesionados_fem','edad_lesionados_masc','edad_lesionados_desc','edad_occiso_fem','edad_occiso_masc','edad_occiso_desc']
for col in cols_edades:
    values = { x:str(np.mean([float(y) for y in x.split('_')])) for x in df_1['edad_lesionados_masc'].unique() if '_' in x } 
    dicc[col] = values
    
df_1 = df_1.replace(dicc)


In [61]:
df_1.loc[df_1['edad_occiso_masc'] == '', 'edad_occiso_masc'] = '0'

In [62]:
for edad1 in cols_edades:
    dicc={}
    edades_unicas = list(df_1[edad1].unique())
    for edad2 in edades_unicas:
        try:
            prueba = float(edad2)
        except:
            dicc[edad2] = '0'
            
    df_1[edad1+'_aux'] = df_1[edad1].map(dicc).astype(float)
    no_nulos = df_1[~(df_1[edad1+'_aux'].isnull())]
    no_nulos = no_nulos.drop([edad1],axis=1)
    no_nulos.rename(columns = {edad1+'_aux': edad1}, inplace=True)
    nulos = df_1[df_1[edad1+'_aux'].isnull()]
    nulos = nulos.drop([edad1+'_aux'], axis=1)
    df_1 = pd.concat([nulos,no_nulos], axis = 0, ignore_index = True)


In [63]:
df_1

,no_folio,fecha_evento,mes,hora,dia,condicion,tipo_de_evento,prioridad,diagnostico_lesionado,diagnostico_occiso,...,algoritm_edad_lesionados,edad_lesionados_fem,edad_lesionados_masc,edad_lesionados_desc,occisos_fem,occisos_masc,occisos_desc,lesionados_fem,lesionados_masc,lesionados_desc
0,2077989,2019-10-12,octubre,6,sabado,OCCISO,ATROPELLADO,ALTA,DESCONOCIDO,DESCONOCIDO,...,"[0, 28_31, 0]",0,29.5,0,DESCONOCIDO,PEATON,DESCONOCIDO,DESCONOCIDO,PEATON_PEATON,DESCONOCIDO
1,2203573,2019-11-18,noviembre,14,lunes,LESIONADO,CHOQUE,BAJA,DESCONOCIDO,DESCONOCIDO,...,"[28, 70, 0]",28,70,0,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,MOTOCICLISTA,PEATON,DESCONOCIDO
2,1937165,2019-09-02,septiembre,1,lunes,OCCISO,CHOQUE,ALTA,DESCONOCIDO,DESCONOCIDO,...,"[0, 23_29, 0]",0,26.0,0,CONDUCTOR,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,PASAJERO_PASAJERO,DESCONOCIDO
3,2002987,2019-09-20,septiembre,22,viernes,OCCISO,CHOQUE,ALTA,DESCONOCIDO,DESCONOCIDO,...,"[0, 33_28_38, 0]",0,33.0,0,DESCONOCIDO,CONDUCTOR,DESCONOCIDO,DESCONOCIDO,PASAJERO_PASAJERO_PASAJERO,DESCONOCIDO
4,1989008,2019-09-16,septiembre,16,lunes,OCCISO,CHOQUE,ALTA,DESCONOCIDO,DESCONOCIDO,...,"[0, 29_82, 0]",0,55.5,0,DESCONOCIDO,PASAJERO,DESCONOCIDO,DESCONOCIDO,MOTOCICLISTA_CONDUCTOR,DESCONOCIDO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91262,C5/20230430/03908,2023-04-30,abril,16,martes,0.0,CHOQUE,NO,0,0,...,"[0, 0, 0]",0,0,0,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,0.0
91263,C5/20230430/06161,2023-04-30,abril,23,lunes,0.0,CHOQUE,NO,0,0,...,"[0, 0, 0.0]",0,0,0.0,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,0.0
91264,C5/20230430/06147,2023-04-30,abril,23,lunes,0.0,VOLCADURA,SI,0,0,...,"[0, 0, 0.0]",0,0,0.0,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,0.0
91265,C5/20230402/02624,2023-04-02,abril,10,lunes,0.0,ATROPELLADO,SI,0,0,...,"[0, 0, 0.0]",0,0,0.0,DESCONOCIDO,0.0,,DESCONOCIDO,DESCONOCIDO,


## -----------------------------------------------------------------------------------------------------------------

In [64]:
select1 = ['no_folio','fecha_evento','year','mes','dia','hora','condicion','tipo_de_evento','punto_1','punto_2','colonia',
         'alcaldia','coordenada_x','coordenada_y','tipo_de_interseccion','tipo_de_vehiculo_1','marca_de_vehiculo_1','color_vehiculo_1','tipo_de_vehiculo_2','marca_de_vehiculo_2','color_vehiculo_2',
         'tipo_de_vehiculo_3','marca_de_vehiculo_3','color_vehiculo_3','identidad','unidad_medica_de_apoyo','hospital','prioridad',
         'interseccion_semaforizada','clasificacion_de_la_vialidad','sentido_de_circulacion',
           'peaton_occiso','peaton_lesionado','ciclista_occiso','ciclista_lesionado','pasajero_occiso','pasajero_lesionado','motociclista_occiso',
           'motociclista_lesionado','conductor_occiso','conductor_lesionado']
          

select2 = ['edad_occiso','se_desconoce_occiso','occisos_femeninos','occisos_masculinos','occiso_se_desconoce','total_occisos','edad_lesionado','se_desconoce_lesionado','lesionados_femeninos','lesionados_masculinos','lesionado_se_desconoce','total_lesionados',
          'occisos_fem','occisos_masc','occisos_desc','lesionados_fem','lesionados_masc','lesionados_desc'] + ['edad_lesionados_fem','edad_lesionados_masc','edad_lesionados_desc','edad_occiso_fem','edad_occiso_masc','edad_occiso_desc']


df_1 = df_1[select1+select2]

In [65]:
condiciones_fem = [(df_1.occisos_femeninos > 0) & (df_1.lesionados_femeninos > 0),
               (df_1.occisos_femeninos > 0) & (df_1.lesionados_femeninos == 0),
               (df_1.occisos_femeninos == 0) & (df_1.lesionados_femeninos > 0)
               ]
condiciones_masc = [(df_1.occisos_masculinos > 0) & (df_1.lesionados_masculinos > 0),
               (df_1.occisos_masculinos > 0) & (df_1.lesionados_masculinos == 0),
               (df_1.occisos_masculinos == 0) & (df_1.lesionados_masculinos > 0)
               ]

condiciones_desc = [(df_1.occiso_se_desconoce > 0) & (df_1.lesionado_se_desconoce > 0),
               (df_1.occiso_se_desconoce > 0) & (df_1.lesionado_se_desconoce == 0),
               (df_1.occiso_se_desconoce == 0) & (df_1.lesionado_se_desconoce > 0)
               ]

elecciones_fem = np.array(('femenino', 'femenino','femenino'), dtype="object")
elecciones_masc = np.array(('masculino', 'masculino','masculino'), dtype="object")
elecciones_desc = np.array(('desconocido', 'desconocido','desconocido'), dtype="object")

df_1["involucrado_fem"] = np.select(condiciones_fem, elecciones_fem, None)
df_1["involucrado_masc"] = np.select(condiciones_masc, elecciones_masc, None)
df_1["involucrado_desc"] = np.select(condiciones_desc, elecciones_desc, None)

C:\Users\XMX5634\AppData\Local\Temp/ipykernel_1956/705145096.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1["involucrado_fem"] = np.select(condiciones_fem, elecciones_fem, None)
C:\Users\XMX5634\AppData\Local\Temp/ipykernel_1956/705145096.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1["involucrado_masc"] = np.select(condiciones_masc, elecciones_masc, None)
C:\Users\XMX5634\AppData\Local\Temp/ipykernel_1956/705145096.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [66]:
#edad = ['edad_occiso','se_desconoce_occiso','edad_lesionado','se_desconoce_lesionado']
fem_ = ['involucrado_fem','occisos_femeninos','lesionados_femeninos','occisos_fem','lesionados_fem','edad_lesionados_fem','edad_occiso_fem']
masc_ = ['involucrado_masc','occisos_masculinos','lesionados_masculinos','occisos_masc','lesionados_masc','edad_lesionados_masc','edad_occiso_masc']
desc_ = ['involucrado_desc','occiso_se_desconoce','lesionado_se_desconoce','occisos_desc','lesionados_desc','edad_lesionados_desc','edad_occiso_desc']

In [67]:
rename_fem = {'involucrado_fem':'involucrado','occisos_femeninos':'total_occisos','lesionados_femeninos':'total_lesionados','occisos_fem':'occiso','lesionados_fem':'lesionado','edad_lesionados_fem':'edad_lesionado','edad_occiso_fem':'edad_occiso'}
rename_masc = {'involucrado_masc':'involucrado','occisos_masculinos':'total_occisos','lesionados_masculinos':'total_lesionados','occisos_masc':'occiso','lesionados_masc':'lesionado','edad_lesionados_masc':'edad_lesionado','edad_occiso_masc':'edad_occiso'}
rename_desc = {'involucrado_desc':'involucrado','occiso_se_desconoce':'total_occisos','lesionado_se_desconoce':'total_lesionados','occisos_desc':'occiso','lesionados_desc':'lesionado','edad_lesionados_desc':'edad_lesionado','edad_occiso_desc':'edad_occiso'}

fem = df_1[~df_1['involucrado_fem'].isnull()].drop(masc_+desc_ ,axis=1)[select1+fem_]
masc = df_1[~df_1['involucrado_masc'].isnull()].drop(fem_+desc_ ,axis=1)[select1+masc_]
desc = df_1[~df_1['involucrado_desc'].isnull()].drop(fem_+masc_ ,axis=1)[select1+desc_]

fem.rename(columns = rename_fem, inplace=True)
masc.rename(columns = rename_masc, inplace=True)
desc.rename(columns = rename_desc, inplace=True)

In [68]:
df_1 = pd.concat([fem,masc,desc], axis = 0, ignore_index = True)

In [69]:
# cuando se haga la limpieza agregar tambien estos campos
drop1 = ['peaton_occiso', 'peaton_lesionado',
       'ciclista_occiso', 'ciclista_lesionado', 'pasajero_occiso',
       'pasajero_lesionado', 'motociclista_occiso', 'motociclista_lesionado',
       'conductor_occiso', 'conductor_lesionado']


#borrar definitivamente desde el inicio
# observaciones: trata de ver como poder etiquetar dichos comentarios
drop_inicio = ['identidad','condicion','tipo_de_vehiculo_3','marca_de_vehiculo_3','color_vehiculo_3','hospital']

In [70]:
df_1.sort_values(["no_folio", "fecha_evento"], inplace=True)
df_1 = df_1.drop(drop1+drop_inicio,axis=1)

In [71]:
df_1 = df_1.fillna({'marca_de_vehiculo_1':'DESCONOCIDO','occiso':'DESCONOCIDO','interseccion_semaforizada':'DESCONOCIDO','lesionado':'DESCONOCIDO',
               'clasificacion_de_la_vialidad':'DESCONOCIDO','sentido_de_circulacion':'DESCONOCIDO','color_vehiculo_2':'DESCONOCIDO',
               'marca_de_vehiculo_2':'DESCONOCIDO','tipo_de_vehiculo_2':'DESCONOCIDO','tipo_de_vehiculo_2':'DESCONOCIDO','color_vehiculo_1':'DESCONOCIDO'})

In [72]:
df = df_1 # la idea es que ahi sea df_1 + df_2

In [73]:
df['fecha_evento'] = pd.to_datetime(df['fecha_evento'])
df['year'] = df['year'].astype('str')
df['mes'] = df['mes'].astype('str')
df['dia'] = df['dia'].astype('str')
df['hora'] = df['hora'].astype('int')
df['tipo_de_evento'] = df['tipo_de_evento'].astype('str')
df['punto_1'] = df['punto_1'].astype('str')
df['punto_2'] = df['punto_2'].astype('str')
df['colonia'] = df['colonia'].astype('str')
df['alcaldia'] = df['alcaldia'].astype('str')
df['coordenada_x'] = df['coordenada_x'].astype('float')
df['coordenada_y'] = df['coordenada_y'].astype('float')
df['total_occisos'] = df['total_occisos'].astype('int')
df['total_lesionados'] = df['total_lesionados'].astype('int')
df['edad_lesionado'] = df['edad_lesionado'].astype('float')
df['edad_occiso'] = df['edad_occiso'].astype('float')

In [75]:
df.to_csv('C:/Users/XMX5634/Documents/Hechos-de-Transito-registrados-por-la-SSC/hechos_transito2.csv', index=False)

# EXPLORACION DE DATOS

In [ ]:
tabla_frecuencias(df,'parte_dia')

In [ ]:
aux1 = pd.DataFrame(df.groupby(['year','mes'])['tipo_de_evento'].count()).reset_index()

In [ ]:
tabla_frecuencias(df,'tipo_de_evento')

Se observa que el 90% de los accidentes de tránsito que ocurren en la CDMX son por causas como CHOQUES, ATROPELLOS Y DERRAPOS, en donde el 56% son provocados por CHOQUES. 

Valdria la pena revisar estas tres categorías por separado y las ultimas 3 en conjunto (CAIDA DE CICLISTA, VOLCADURA Y CAIDA DE PASAJERO) ya que aproximadamente el 5% del total, son accidentes de este estilo. 

## Análisis Urbano
### de lo general a lo particular

In [ ]:
aux = pd.DataFrame(df.groupby(['alcaldia','tipo_de_evento'])['total_occisos','total_lesionados'].sum()).reset_index()

In [ ]:
plt.figure(figsize = (15,10))
sns.barplot(y = 'alcaldia', x = 'total_occisos',hue='tipo_de_evento',data=aux.sort_values('total_occisos', ascending = False))
plt.title('Total de Muertos por Alcaldía')

In [ ]:
plt.figure(figsize = (15,10))
sns.barplot(y = 'alcaldia', x = 'total_lesionados',hue='tipo_de_evento', data=aux.sort_values('total_lesionados', ascending = False))
plt.title('Total de Lesionados por Alcaldía')

In [ ]:
aux = pd.DataFrame(df.groupby(['alcaldia','involucrado','tipo_de_evento'])['total_occisos','total_lesionados'].sum()).reset_index()

In [ ]:
plt.figure(figsize = (15,10))
sns.barplot(y = 'alcaldia', x = 'total_occisos',hue='involucrado',data=aux.sort_values('total_occisos', ascending = False))

In [ ]:
plt.figure(figsize = (15,10))
sns.barplot(y = 'alcaldia', x = 'total_lesionados',hue='involucrado',data=aux.sort_values('total_lesionados', ascending = False))

Analisis:

Lesionados:
* Cuauhtemoc, Iztapalapa y Gustavo A. Madero las alcaldias que mas lesiones presentan.
* De manera general en todas alcaldias, ocurren mas las lesiones por CHOQUES.
* Atropellos y Derrapos suelen ocurrir con la misma frecuencia. 
* Caida de Ciclista,Volcadura y Caida de Pasajero ocurren de igual manera con la misma frecuencia y en menores proporciones
* Suelen lesionarse mas Hombres que Mujeres en todas las alcaldias. 

Muertos:
* Tlalpan, Iztapalapa y Gustavo A. Madero las alcaldias que mas muertes presentan.
* Las muertes por atropellos sobrepasan en Venustiano Carranza e Iztacalco.
* Suelen morir mas Hombres que Mujeres en todas las alcaldias.

**target 1 = ¿Cuales son las condiciones en las que mueren y se lesionan mas los Hombres por choques en Iztapalapa, Gustavo A.Madero, Tlalpan y Cuauhtemoc?**

**target 2 = ¿Cuales son las condiciones por las que mueren mas Hombres atropellados en Venustiando Carranza e Iztacalco?**

In [ ]:
targ_1 = df[(df['involucrado'] == 'masculino') & (df['tipo_de_evento'] == 'CHOQUE') & (df['alcaldia'].isin(['GUSTAVO A MADERO','TLALPAN','CUAUHTEMOC','IZTAPALAPA'])) ]

In [ ]:
aux = pd.DataFrame(targ_1.groupby(['alcaldia','colonia'])['total_occisos','total_lesionados'].sum()).reset_index()

In [ ]:
lesionados_ = []
occisos_ = []
for alc in aux.alcaldia.unique():
    lesionados = aux[aux['alcaldia'] == alc][['alcaldia','colonia','total_lesionados']].sort_values('total_lesionados', ascending = False)
    les = lesionados.iloc[0:10,:]
    occisos = aux[aux['alcaldia'] == alc][['alcaldia','colonia','total_occisos']].sort_values('total_occisos', ascending = False)
    occs = occisos.iloc[0:10,:]
    lesionados_.append(les)
    occisos_.append(occs)
    
df_les = pd.concat(lesionados_, axis = 0, ignore_index = True).sort_values('total_lesionados')
df_occi = pd.concat(occisos_, axis = 0, ignore_index = True).sort_values('total_occisos')

In [ ]:
plt.figure(figsize = (15,10))
sns.lineplot(x='total_lesionados', y='colonia', data = df_les, color='red')

* Juan Escutia, Roma Sur, Leyes de Reforma 3a Secc, Sta Maria la Ribera, Guerrero, Obrera, Roma Nte, Doctores y Centro son todas colonias donde los hombres se lesionan mas por CHOQUES > 200.

In [ ]:
plt.figure(figsize = (15,10))
sns.lineplot(x='total_occisos', y='colonia', data = df_occi, color='black')

* Leyes de Reforma 3a Seccion, El Paraiso, Morelos, San Felipe de Jesus, Juan Escutia, Area Fed Central de Abastos, San Andres Totoltepec, San Miguel Topilejo, Centro, Rusticia Tlalpan y Doctores son las colonias en donde mas hombres mueren  a causa de un choque > 5. 

In [ ]:

aux = pd.DataFrame(targ_1.groupby(['occiso'])['total_occisos'].sum()).reset_index()

In [ ]:
plt.figure(figsize=(10,10))
colors = sns.color_palette('bright')
plt.pie(aux.total_occisos, labels=aux.occiso ,colors = colors, autopct = '%0.0f%%')
plt.title('Indentidad de las personas involucradas en las muertes por CHOQUES en CDMX')
plt.show()

Los pasajeros son quienes mas fallecen por CHOQUES en CDMX correspondiente a un 12%, seguido de los Motoclistas con un 10% y en tercer lugar los conductores con un 7% 

In [ ]:
aux = pd.DataFrame(targ_1.groupby(['lesionado'])['total_lesionados'].sum()).reset_index()

In [ ]:
aux

In [ ]:
plt.figure(figsize=(10,10))
colors = sns.color_palette('bright')
plt.pie(aux.total_lesionados, labels=aux.lesionado ,colors = colors, autopct = '%0.0f%%')
plt.title('Indentidad de las personas involucradas en lesiones por CHOQUES en CDMX')
plt.show()

Los Motociclistas son las personas que mas se lesionan por CHOQUES, correspondiente al 18% de lesiones, seguido de los Conductores con un 12% y al finaal los Pasajores con un 10%. 


In [ ]:
aux = targ_1[targ_1['colonia'].isin(df_les.colonia.unique())]

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(y = 'colonia', x = 'hora', data=aux)
plt.title('Distribución de Horas por Colonia')

Se observa que la hora en la que suceden los sucesos por CHOQUES (Muertos y Lesionados), suelen ser muy similares en todas las alcaldías, incluso tomando las horas mínimas y máximas. 

Todas poseen una mediana entre las 12 y las 16 horas, el 50% de los sucesos de cada alcaldía ocurren enntre casi las mismas horas. 

# PREPROCESAMIENTO DE DATOS

### Ordinal Encode

In [ ]:
dicc_mes = {'ENERO':1,'FEBRERO':2,'MARZO':3,'ABRIL':4,'MAYO':5,'JUNIO':6,'JULIO':7,'AGOSTO':8,'SEPTIEMBRE':9,'OCTUBRE':10,'NOVIEMBRE':11,'DICIEMBRE':12}
df['mes_aux'] = df['mes'].map(dicc_mes)
df = df.drop(columns = ['mes'], axis=1)
df.rename(columns = {'mes_aux':'mes'}, inplace=True)

In [ ]:
df['dia_aux'] = df['dia'].map(dict(zip(['LUNES','MARTES','MIERCOLES','JUEVES','VIERNES','SABADO','DOMINGO'],[i for i in range(1,8)])))
df = df.drop(columns = ['dia'], axis=1)
df.rename(columns = {'dia_aux':'dia'}, inplace=True)

In [ ]:
df['prioridad_aux'] = df['prioridad'].map({'BAJA':1,'MEDIA':2, 'ALTA':3})
df = df.drop(columns = ['prioridad'], axis=1)
df.rename(columns = {'prioridad_aux':'prioridad'}, inplace=True)

In [ ]:
fechas = df['year'].astype(str) + "-" + df['mes'].astype(str) + "-" + '01'
df['fecha_mensual'] = pd.to_datetime(fechas)

### Dummies

In [ ]:
cols_dummies = ['tipo_de_evento','alcaldia','involucrado','interseccion_semaforizada']

In [ ]:
df_dummies = pd.get_dummies(data=df,columns=cols_dummies)

### Frecuency Encoding 

In [ ]:
cols_frecuency = ['colonia','tipo_de_interseccion','unidad_medica_de_apoyo','occiso','lesionado','tipo_de_vehiculo_1',
                 'marca_de_vehiculo_1','color_vehiculo_1','tipo_de_vehiculo_2','marca_de_vehiculo_2','color_vehiculo_2',
                 'clasificacion_de_la_vialidad','punto_1','punto_2']

In [ ]:
names = []
for col in cols_frecuency:
    encoder_2 = ce.CountEncoder(cols=col, normalize=True)
    name_frecuency = col + '_frequency_encouded'
    df_dummies[name_frecuency] = encoder_2.fit_transform(df_dummies[col])
    names.append(name_frecuency)
    df_dummies = df_dummies.drop([col], axis=1)

In [ ]:
# borrar def al final 
drop_final = ['no_folio','fecha_evento','year','parte_dia','sentido_de_circulacion','fecha_mensual']
df_dummies = df_dummies.drop(drop_final,axis=1)

### ACP

In [ ]:
# Inicializar PCA y ajustar a los datos
pca = PCA().fit(df_dummies)

# Obtener la varianza explicada de cada componente principal
variance = pca.explained_variance_ratio_

# Obtener la varianza acumulada de los componentes principales
cumulative_variance = np.cumsum(variance)

# Graficar la varianza acumulada
plt.figure(figsize=(20,10)) 
plt.plot(cumulative_variance)
plt.xlabel('Número de componentes principales')
plt.xlim(1,pca.n_components_)
plt.ylabel('Varianza acumulada')
plt.show()

In [ ]:
# Componente principal asociado al 84% de la varianza explicada. 
i = 1
dicc = {}
for cum_var in cumulative_variance:
    if 0.85 <= cum_var:
        dicc[i] = cum_var
        break
    i += 1
print(dicc)

In [ ]:
pca = PCA(n_components = 2)
principal_comp = pca.fit_transform(df_dummies)
principal_comp

In [ ]:
dicc = {}
for i in range(1,3):
    pca = 'PCA'+str(i)
    dicc[pca] = principal_comp[:,i-1]
    
df_pca = pd.DataFrame(dicc)

In [ ]:
df_pca

In [ ]:
df_pca.to_csv('C:/Users/XMX5634/Documents/Hechos-de-Transito-registrados-por-la-SSC/pca.csv',index=False)

In [ ]:
np.random.seed(10)
train, test = train_test_split(df_pca, train_size = 0.8, random_state = 0)

In [ ]:
train.to_csv('C:/Users/XMX5634/Documents/Hechos-de-Transito-registrados-por-la-SSC/pca_train.csv',index=False)

In [ ]:
test.to_csv('C:/Users/XMX5634/Documents/Hechos-de-Transito-registrados-por-la-SSC/pca_test.csv',index=False)

In [ ]:
#aplicamos ahora si el metodo del codo. 
scores_1 = [] #aqui acumularemos todas las sumas al cuadrado.  
range_values = range(1,20)

for i in range_values: 
    kmeans = KMeans(n_clusters = i) #lo que hace esto es crear el modelo de k-means, para el dataset normalizado, junto 
    kmeans.fit(train) #con esta linea, para cada i, hará su propio cluster de k-means. 
    scores_1.append(kmeans.inertia_) #una vez completados todos ellos, lo que interesa es la suma de los cuadrados intracluster
                                    #parametro, llamado inertia.
plt.figure(figsize = (15,10))
plt.plot(range_values, scores_1, 'bx-')
plt.title("Encontrar el número optimo de Clusters")
plt.xlabel("Clusters")
plt.ylabel("WCSS(k)")
plt.show()

## Aplicar K-Means

In [ ]:
kmeans = KMeans(4) #numero de clusters
kmeans.fit(df_pca)
labels = kmeans.labels_

In [ ]:
labels

In [ ]:
kmeans.cluster_centers_.shape 

In [ ]:
kmeans.cluster_centers_ # Centroides de cada clase. 

In [ ]:
#si nos quedamos con el centros y visualizarnos de una forma mas comoda. 
clusters_centers = pd.DataFrame(data = kmeans.cluster_centers_, columns = [df_pca.columns])
clusters_centers
#para cada uno de los 6 clusters, desde el cluster 0 hasta el 5, podemos ver donde estaria el centro, cada fila representa uno 
#de los centroides y en cada columna donde se encuentra la coordenada en cuestión.

#podemos fijarnos un poco en cuales son las tendencias, 

In [ ]:
#si nos quedamos con el centros y visualizarnos de una forma mas comoda. 
clusters_centers = pd.DataFrame(data = kmeans.cluster_centers_, columns = [df_pca.columns])
clusters_centers
#para cada uno de los 8 clusters, desde el cluster 0 hasta el 7, podemos ver donde estaria el centro, cada fila representa uno 
#de los centroides y en cada columna donde se encuentra la coordenada en cuestión.

#podemos fijarnos un poco en cuales son las tendencias, 

In [ ]:
pca_df = pd.concat([df_pca, pd.DataFrame({'cluster':labels})], axis = 1)

In [ ]:
pca_df

In [ ]:
plt.boxplot(df_dummies['edad_lesionado'])

In [ ]:
plt.figure(figsize=(10,10))
ax = sns.scatterplot(x = "PCA1", y = "PCA2", hue = "cluster", data = pca_df, 
                    palette = ["red", "green", "blue", "pink"])
plt.show()
#todos los puntos del mismo color, pertenecen al mismo cluster. 
#el kmeans no ha sido aplicado a este espacio bidimensional, no a las cp, sino a las obs originales, una vez aplicadas
#solo proyectamos en un espacio de dimension 2.

# Conclusiones

In [ ]:
df_dummies['cluster'] = list(pca_df['cluster'])

In [ ]:
df_dummies